In [1]:
from networks.models import build_model

In [2]:
import torch
import cv2
import matplotlib.pyplot as plt

In [12]:
model_custom = build_model("FBA.pth", custom_groupnorm=True).cuda().eval()
model = build_model("FBA.pth", custom_groupnorm=False).cuda().eval()

modifying input layer to accept 11 channels
modifying input layer to accept 11 channels


In [9]:
image = torch.randn((1, 3, 128, 128)).cuda()
two_chan_trimap=  torch.randn((1, 2, 128, 128)).cuda()
image_n=torch.randn((1, 3, 128, 128)).cuda()
trimap_transformed =torch.randn((1, 6, 128, 128)).cuda()
output = model(image, two_chan_trimap, image_n, trimap_transformed)

In [14]:
output = model(image, two_chan_trimap, image_n, trimap_transformed)
output_custom = model_custom(image, two_chan_trimap, image_n, trimap_transformed)
print(torch.mean( (output - output_custom)**2).item())

5.12290210252786e-09


In [26]:
jitmodel = torch.jit.trace(model, [image, two_chan_trimap, image_n, trimap_transformed])
jitmodel_custom = torch.jit.trace(model_custom, [image, two_chan_trimap, image_n, trimap_transformed])

In [27]:
output_jit = jitmodel(image, two_chan_trimap, image_n, trimap_transformed)
output_jit_custom = jitmodel_custom(image, two_chan_trimap, image_n, trimap_transformed)

In [28]:
print(torch.mean( (output - output_jit)**2).item())
print(torch.mean( (output_custom  - output_jit_custom )**2).item())

0.0
0.0


In [23]:
%timeit _ = jitmodel(image, two_chan_trimap, image_n, trimap_transformed); torch.cuda.synchronize()

5.9 ms ± 73 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%timeit _ = model(image, two_chan_trimap, image_n, trimap_transformed); torch.cuda.synchronize()

8.08 ms ± 80 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
%timeit _ = model_custom(image, two_chan_trimap, image_n, trimap_transformed); torch.cuda.synchronize()

13 ms ± 49.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
%timeit _ = jitmodel_custom(image, two_chan_trimap, image_n, trimap_transformed); torch.cuda.synchronize()

9.03 ms ± 55.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
